<a href="https://colab.research.google.com/github/CarlosMendez1997Col/Automation_of_satellite_image_downloads_GeeMap_APIs_REST/blob/main/1.%20Sentinel/Automatizar_descarga_Sentinel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Automated script to download Sentinel-1 images from a Google Earth Engine using GeeMap`



# Import libraries and packages

In [1]:
!pip install geemap # Install the geemap library for interactive mapping with Earth Engine
!pip install earthengine-api # Install the Earth Engine Python API

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.9 MB/s eta 0:00:00


In [2]:
import ee # Import the Earth Engine API
import geemap # Import geemap for visualization and mapping
import webbrowser # Import webbrowser to open authentication URLs
import ipywidgets as widgets # Import ipywidgets for interactive widgets in Jupyter notebooks
from datetime import datetime # Import datetime for handling dates

## Autentication in Google Colab and GEE

In [3]:
# --- Authenticate and initialize Earth Engine ---
auth_url = ee.Authenticate(auth_mode='notebook') # Generate authentication URL for notebook mode
webbrowser.open(auth_url) # Open the authentication URL in the browser
ee.Authenticate() # Perform authentication
ee.Initialize() # Initialize the Earth Engine session

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/drive%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=F8On2SlN9zcUD7ena-lnRyE13EfSXReau072WKg29s8&tc=b0FkcG9D9GGkgbtgnAIqcIW7xe1oBCzn4NAPhoUHMKc&cc=Y9Qyvnfr4UqFjvrFWFc_Wzs6zP3rgy1b59AwXHoYkh8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ASc3gC27E3vZ8oXfvKP0kJf1k7JRMYVzzPsdFyXfOsSp5vqWtMjuxks6oVw

Successfully saved authorization token.


## Create basemaps and import Area Of Interest (AOI)

In [4]:
# --- Load Cobija shapefile from Earth Engine assets ---
cobija_fc = ee.FeatureCollection("projects/gee-projects-481514/assets/CobijaSHP")

# --- Create a map and add Cobija shapefile ---
MapCobija = geemap.Map() # Create a geemap Map object
MapCobija.addLayer(cobija_fc, {"color": "darkblue", "opacity": 0.7}, "Cobija SHP") # Add Cobija shapefile layer
MapCobija.centerObject(cobija_fc, 13) # Center the map on Cobija with zoom level 13
MapCobija.setOptions('HYBRID') # Set basemap style to hybrid (satellite + labels)

# --- Define available basemap styles ---
styles = {
    "OpenStreetMap": "OpenStreetMap",
    "Esri World Street Map": "Esri.WorldStreetMap",
    "Esri World Imagery": "Esri.WorldImagery",
    "Esri World Topo Map": "Esri.WorldTopoMap",
    "Esri World Gray Canvas": "Esri.WorldGrayCanvas",
    "Esri World Shaded Relief": "Esri.WorldShadedRelief",
    "Esri World Terrain": "Esri.WorldTerrain",
    "CartoDB DarkMatter": "CartoDB.DarkMatter",
    "CartoDB Positron": "CartoDB.Positron",
    "CartoDB Voyager": "CartoDB.Voyager",
    "OpenTopoMap": "OpenTopoMap"
}

# --- Create dropdown widget to select basemap style ---
style_dropdown = widgets.Dropdown(
    options=list(styles.keys()), # Options are the keys of the styles dictionary
    value="Esri World Imagery", # Default selected basemap
    description="Basemap:" # Label for the dropdown
)

# --- Function to update basemap style when dropdown changes ---
def update_style(change):
    MapCobija.add_basemap(styles[change['new']]) # Add new basemap selected by user
    MapCobija.addLayer(cobija_fc, {"color": "darkblue", "opacity": 0.7}, "Cobija SHP") # Re-add Cobija layer
    MapCobija.centerObject(cobija_fc, 13) # Re-center map on Cobija

style_dropdown.observe(update_style, names='value') # Observe changes in dropdown value

display(style_dropdown) # Display the dropdown widget
MapCobija # Display the map

Dropdown(description='Basemap:', index=2, options=('OpenStreetMap', 'Esri World Street Map', 'Esri World Image…

Map(center=[-11.031617083854373, -68.77127014491582], controls=(WidgetControl(options=['position', 'transparen…

# Search Sentinel Images in the AOI

## Sentinel-1 SAR GRD: C-band Synthetic Aperture Radar Ground Range Detected

In [5]:
# --- Load Sentinel-1 GRD image collection ---
s1_collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(cobija_fc) # Filter by Cobija region

# --- Get start and end dates for filtering ---
first_image = s1_collection.sort('system:time_start').first() # Get earliest image
start_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo() # Extract start date
end_date = datetime.utcnow().strftime("%Y-%m-%d") # Use current date as end date

# --- Filter Sentinel-1 collection by parameters ---
s1 = (s1_collection
      .filterDate(start_date, end_date) # Filter by date range
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) # Require VV polarization
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) # Require VH polarization
      .filter(ee.Filter.eq('instrumentMode', 'IW')) # Use Interferometric Wide swath mode
      .filter(ee.Filter.eq('productType', 'GRD')) # Use GRD product type
      .filter(ee.Filter.eq('platform_number', 'A')) # Use Sentinel-1A platform
      .sort('system:time_start', False)) # Sort by date descending

# --- Additional orbit filters ---
orbit_filter_pass = 'DESCENDING' # Orbit direction filter
relative_orbit = 127 # Relative orbit number filter

if orbit_filter_pass is not None:
    s1 = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit_filter_pass)) # Apply orbit pass filter

if relative_orbit is not None:
    s1 = s1.filter(ee.Filter.eq('relativeOrbitNumber_start', relative_orbit)) # Apply relative orbit filter

## Additional processes in S1 Images


In [6]:
# --- Function to remove border noise ---
def remove_border_noise(image):
    mask_vv = image.select('VV').mask() # Get mask for VV band
    mask_vh = image.select('VH').mask() # Get mask for VH band
    combined_mask = mask_vv.And(mask_vh) # Combine masks
    return image.updateMask(combined_mask) # Apply combined mask to image

# --- Function to apply Lee speckle filter ---
def lee_filter(image):
    def lee_band(band): # Inner function to filter a single band
        band_img = image.select(band) # Select band
        mean = band_img.reduceNeighborhood(reducer=ee.Reducer.mean(), kernel=ee.Kernel.square(3)) # Local mean
        variance = band_img.reduceNeighborhood(reducer=ee.Reducer.variance(), kernel=ee.Kernel.square(3)) # Local variance
        sample_var = variance.divide(mean.multiply(mean)) # Compute coefficient of variation
        weight = sample_var.divide(sample_var.add(1)) # Compute adaptive weight
        filtered = mean.add(weight.multiply(band_img.subtract(mean))) # Apply filter
        return filtered.rename(band + "_filtered") # Rename filtered band
    vv_filtered = lee_band('VV') # Apply filter to VV band
    vh_filtered = lee_band('VH') # Apply filter to VH band
    return image.addBands([vv_filtered, vh_filtered]) # Add filtered bands to image

## Apply additional processes

In [7]:
# --- Apply preprocessing functions to entire collection ---
s1_preprocessed = (s1
                   .map(remove_border_noise) # Apply border noise removal
                   .map(lee_filter)) # Apply speckle filter

## Print S1 images in AOI

In [8]:
# --- Create list of images and extract metadata ---
images_list = s1_preprocessed.toList(s1_preprocessed.size()) # Convert collection to list
info_list = []
for i in range(s1_preprocessed.size().getInfo()): # Loop through all images
    img = ee.Image(images_list.get(i)) # Get image
    info = img.getInfo() # Get image metadata
    info_list.append({
        'ID': info['id'], # Image ID
        'Fecha': info['properties']['system:time_start'], # Acquisition date
        'Polarización': info['properties']['transmitterReceiverPolarisation'] # Polarization info
    })

print("Imágenes Sentinel-1 preprocesadas disponibles en Cobija:")
for info in info_list:
    print(info) # Print metadata for each image

Imágenes Sentinel-1 preprocesadas disponibles en Cobija:
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20260108T101431_20260108T101501_062674_07DBA5_DF0D', 'Fecha': 1767867271000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251227T101432_20251227T101502_062499_07D4E7_D566', 'Fecha': 1766830472000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251215T101433_20251215T101503_062324_07CE1A_87D7', 'Fecha': 1765793673000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251203T101435_20251203T101504_062149_07C742_86A4', 'Fecha': 1764756875000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251121T101436_20251121T101506_061974_07C06E_317E', 'Fecha': 1763720076000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251109T101436_20251109T101506_061799_07B9A0_2B70', 'Fecha': 1762683276000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_

## Visualize latest 3 images of S1 in the AOI

In [9]:
# --- Limit to last 3 images for visualization ---
s1_10 = s1_preprocessed.limit(3)
images_list_10 = s1_10.toList(s1_10.size())

# --- Create map for visualization ---
MapCobijaS1 = geemap.Map()
for i in range(s1_10.size().getInfo()): # Loop through last 3 images
    img = ee.Image(images_list_10.get(i)) # Get image
    date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo() # Format date
    MapCobijaS1.addLayer(img.select('VV'), {"min": -25, "max": 0}, f"VV {date_str}") # Add VV band
    MapCobijaS1.addLayer(img.select('VV_filtered'), {"min": -25, "max": 0}, f"VV filtered {date_str}") # Add filtered VV
    MapCobijaS1.addLayer(img.select('VH'), {"min": -30, "max": 0}, f"VH {date_str}") # Add VH band
    MapCobijaS1.addLayer(img.select('VH_filtered'), {"min": -30, "max": 0}, f"VH filtered {date_str}") # Add filtered VH

MapCobijaS1.addLayer(cobija_fc, {"color":"darkblue", "opacity":0.7}, "Cobija SHP") # Add Cobija shapefile
MapCobijaS1.centerObject(cobija_fc, 12) # Center map on Cobija
MapCobijaS1 # Display map

Map(center=[-11.031617083854373, -68.77127014491582], controls=(WidgetControl(options=['position', 'transparen…